# Conv1D input layer


- Sliding Window Data 생성

- Conv1D + LSTM model 작성

- Learning Rate Tunning

- Window Size Tuning

### Sunspots 데이터셋을 활용하여 window_dataset 만들기

Data 시각화

- 시간 절약을 위해 최근 1000 개월치 data 만 이용

### windowed dataset 생성

1. series를 이용해 tf.data.Dataset.from_tensor_slices로 데이터셋을 생성합니다.  
2. ds.window를 사용하여 입력 데이터셋을 지정한 윈도우 크기와 함께 슬라이딩 윈도우 방식으로 나눕니다.   drop_remainder=True는 윈도우 크기보다 작은 마지막 윈도우를 제거합니다.  
3. ds.flat_map을 사용하여 각 윈도우를 하나의 배치로 펼칩니다.  
4. ds.shuffle을 사용하여 데이터셋을 무작위로 섞습니다. 이렇게 하면 모델이 훈련 데이터의 순서를 기억하는 것을 방지할 수 있습니다.  
5. ds.map에서 입력 데이터의 마지막 원소를 제외한 부분을 X로 사용하고, 첫 원소를 제외한 부분을 y로 사용하여 다중 스텝 출력을 준비합니다.  
6. 마지막으로, ds.batch를 사용하여 지정된 배치 크기로 데이터를 나누고, ds.prefetch를 사용하여 효율적인 학습을 위해 데이터를 미리 가져옵니다.

#### 예측을 위한 보조 함수

- 훈련이 끝난 후에는 시계열 데이터를 모델이 처리할 수 있는 형태로 변환해서 예측을 진행해야 합니다. 이 때 데이터 배치 생성이나 셔플링은 필요하지 않습니다.
- 우리는 예측을 위해 지도학습에서 사용하는 레이블이 필요하지 않으므로, 데이터 윈도우 크기를 조정할 때 1을 더할 필요가 없습니다.

### train/validation set 분리

## Conv1D + LSTM + Bidirectional

- Causal Convolution(인과관계 컨벌루션)은 시간 t의 출력이 시간 t - 1의 입력에서만 파생되도록 합니다.

- Causal padding 은 `Conv1D`에만 지원됩니다.

<img src="https://user-images.githubusercontent.com/24662250/29258799-9b3ff514-80f7-11e7-83e4-ab701fdc2720.png" width=500 />

인과 관계 컨벌루션은 시간 순서를 중요시하는 시계열 데이터를 다룰 때 쓰는 특별한 컨벌루션 방법입니다. 이 방법을 쓰면, 특정 시간의 결과를 계산할 때 그 시간 이후의 데이터는 사용되지 않고, 오직 그 시간과 그 이전의 데이터만 사용됩니다. 이는 결과가 과거와 현재의 정보에만 기반하도록 하여, 데이터 사이의 자연스러운 인과 관계를 보존합니다.

패딩은 컨벌루션을 수행하기 전에 데이터 주변에 특정 값을 추가하는 과정을 말합니다. 인과 관계 컨벌루션에서 사용되는 'causal padding'은 데이터의 앞부분(과거)에만 패딩을 추가하는 방법입니다. 이렇게 함으로써, 컨벌루션 필터가 데이터의 뒷부분(미래)을 '보지 못하게' 하여, 미래 데이터를 참조하지 않도록 합니다. 쉽게 말해, 예측을 할 때 현재 시점에서 아직 일어나지 않은 일을 참고하지 않도록 하는 것입니다.

TensorFlow에서는 이런 인과 관계 컨벌루션을 구현할 때 `Conv1D` 레이어와 `padding='causal'` 옵션을 사용합니다. 이 옵션을 설정하면, 모델이 과거 데이터만을 참고하여 현재와 미래를 예측하도록 만들어

예를 들어, 시간 스텝에 따른 데이터 'abcde'에 인과 관계 패딩을 적용하면, 'aaabcde'와 같이 됩니다. 여기서 'a'는 과거 데이터를 의미하며, 필터는 'd'와 'e'를 현재 및 미래의 스텝으로 볼 때 이들을 참조하지 않습니다.줍니다.

- 초기 epoch 100 동안 learning rate를 변경해 가며 loss가  증가하지 않는 가장 큰 learning rate 를 찾아냅니다.

- 안정적 상태에서 최소 mae 를 얻은 learning rate 선택

이 코드에서 idx는 주어진 history.history["loss"] 리스트에서 최소 손실 값의 위치를 찾아 리스트 형태로 반환합니다. 리스트에는 최소 손실 값에 해당하는 위치가 True로 표시되고 나머지 위치는 False로 표시됩니다.

### Hyper Parameter Tuning

- optimal learning rate로 충분한 epoch train   
    - optimal lr <-- best_lr  
    - epochs <-- 500

- kernel size를 3으로 조정

- Lambda layer 를 `x * 200`으로 조정

훈련 과정에서 계산된 평균 절대 오차(MAE)와 손실 값(Loss)을 시각화 합니다.

X_test data 를 이용하여 forecast

### True value와 prediction value 비교 시각화

X_test(실제 값)와 forecast(예측 값)을 시계열 데이터로 시각화하는 코드입니다.

plt.plot() 함수를 사용하여 time_test와 X_test(실제 값)를 그래프로 그리고, 범례 레이블을 'True'로 설정합니다.  
plt.plot() 함수를 사용하여 time_test의 끝부분에서 예측된 값의 길이만큼과 forecast(예측 값)를 그래프로 그리고, 범례 레이블을 'Forecast'로 설정합니다.  

초반 window_size 만큼의 데이터는 예측 과정에서 skip되므로, 그래프에서도 그 부분이 생략되어 그려집니다.

X_test와 forecast 간의 평균 제곱 오차(mean squared error, MSE)와 평균 절대 오차(mean absolute error, MAE)를 계산하고 출력합니다.

## Window Size를 바꾸어 가며 Tuning

11 년 주기의 noise 가 심한 data 이므로 다양한 size 의 window 를 try 하며 model tuning --> 132, 30, 64 등